In [14]:
import torch
from rich import print
import einops
import math

In [15]:
x = einops.repeat(torch.arange(20), "length -> batch length", batch=3)

In [16]:
DENOISING_TOKEN = 100
MASK_TOKEN = -1

In [17]:
@torch.no_grad()
def mask_spans(
    sequence: torch.Tensor,
    mask_width: int,
    num_masks: int,
    max_percentage_masked: float,
    causal: bool = False,
) -> tuple[torch.Tensor, torch.Tensor]:
    if causal:
        targets = torch.zeros_like(sequence).copy_(sequence)
    else:
        targets = sequence.roll(1)
        targets[:, 0] = DENOISING_TOKEN

    num_masks = min(num_masks, math.floor(max_percentage_masked * sequence.shape[-1] / mask_width))

    batch_size, seq_length = sequence.shape
    mask = torch.randint(0, seq_length - mask_width + 1, (batch_size, num_masks))
    
    # Create a tensor to hold the mask
    mask_tensor = torch.zeros_like(sequence)
    
    # Create a range for the mask width
    width_range = torch.arange(mask_width).unsqueeze(0).unsqueeze(0)  # Shape: (1, 1, mask_width)
    
    # Expand the mask to the desired width
    mask_positions = mask.unsqueeze(-1) + width_range  # Shape: (batch_size, num_masks, mask_width)
    
    # Clip the positions to stay within sequence bounds
    mask_positions = mask_positions.clamp(0, seq_length - 1)
    
    # Scatter the mask positions into the mask tensor
    mask_tensor.scatter_(1, mask_positions.view(batch_size, -1), 1)

    # Apply the mask to the sequence
    inputs = sequence.masked_fill(mask_tensor.bool(), MASK_TOKEN)

    # Add task token
    inputs = inputs.roll(1, dims=-1)
    inputs[:, 0] = DENOISING_TOKEN

    return inputs, targets

inputs, targets = mask_spans(x, 4, 2, 0.8, True)
print(f"{inputs=}\n{targets=}\n{x=}")

inputs=tensor([[100,   0,   1,   2,   3,   4,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  12,
          13,  14,  15,  16,  17,  18],
        [100,   0,   1,   2,   3,   4,   5,   6,   7,   8,  -1,  -1,  -1,  -1,
          13,  14,  -1,  -1,  -1,  -1],
        [100,   0,   1,   2,   3,   4,   5,  -1,  -1,  -1,  -1,  10,  -1,  -1,
          -1,  -1,  15,  16,  17,  18]])
targets=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]])
x=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]])

In [24]:
@torch.no_grad()
def get_s_denoised_data(
        sequence: torch.Tensor,
        mask_width: int | None,
        masking_rate: float = 0.5,
        causal: bool = False,
) -> tuple[torch.Tensor, torch.Tensor]:
    if causal:
        targets = torch.zeros_like(sequence).copy_(sequence)  # copy sequence to not have negative downstream effects
    else:
        targets = sequence.roll(1)
        targets[:, 0] = DENOISING_TOKEN

    mask_width = mask_width or math.floor(masking_rate * sequence.shape[-1])
    mask_width = min(mask_width, math.floor(masking_rate * sequence.shape[-1]))
    inputs = sequence.roll(1, dims=-1)
    inputs [:, -mask_width:] = MASK_TOKEN
    inputs[:, 0] = DENOISING_TOKEN

    return inputs, targets

inputs, targets = get_s_denoised_data(x, None, 0.25, True)
print(f"{inputs=}\n{targets=}\n{x=}")

inputs=tensor([[100,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
          13,  -1,  -1,  -1,  -1,  -1],
        [100,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
          13,  -1,  -1,  -1,  -1,  -1],
        [100,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
          13,  -1,  -1,  -1,  -1,  -1]])
targets=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]])
x=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19]])